# Agenda

1. Functions
    - Inner functions
    - Dispatch tables
    - Bytecodes and compilation
2. Mypy + annotations
    - Type annotations / type hints
    - Mypy 
    - Modern techniques
3. Objects
    - What happens when we create an object?
    - Attributes (ICPO)
    - Methods 
    - Magic methods
    - Inheritance

# Inner functions

1. Functions are objects (can be arguments to functions, can be return values from functions)
2. When we use `def`, we (a) create a function object and (b) assign to a variable
3. When we assign to a variable in a function, the variable is local

In [1]:
def outer():
    def inner():
        print('I am in inner!')
    return inner

x = outer()    

In [2]:
type(x)

function

In [3]:
x.__name__

'inner'

In [4]:
x()

I am in inner!


In [5]:
def hello():
    return 'Hello!'

In [6]:
hello.__name__

'hello'

In [7]:
y = hello

In [8]:
y.__name__

'hello'

In [9]:
del(hello)

In [10]:
y()

'Hello!'

In [12]:
def outer(a):
    def inner(b):
        print(f'I am in inner {a=}, {b=}!')
    return inner

x = outer(10)

In [13]:
type(x)

function

In [14]:
x.__code__.co_varnames

('b',)

In [15]:
x.__code__.co_argcount

1

In [16]:
x(20)

I am in inner a=10, b=20!


In [17]:
y = outer(15)

In [18]:
y(20)

I am in inner a=15, b=20!


In [19]:
# closure

# Exercise: Password creator creator

1. Define a function, `create_password_creator`, that takes a string argument -- the characters you want in a potential password.
2. The function returns a new function, which takes an int argument.
3. When we call the returned function, we get a new password of the length we said, with random characters from the string we passed.

Example:

    create_number_pw = create_password_creator('12345')
    create_symbol_pw = creator_password_creator('!@#$%')

    new_number_pw = create_number_pw(9)
    new_symbol_pw = create_symbol_pw(11)

You can use `random.choice`    

In [21]:
import random

def create_password_creator(s):
    def create_password(n):
        output = ''
        for index in range(n):
            output += random.choice(s)
        return output
    return create_password

create_number_pw = create_password_creator('12345')
create_symbol_pw = create_password_creator('!@#$%')

new_number_pw = create_number_pw(9)
new_symbol_pw = create_symbol_pw(11)

print(new_number_pw)
print(new_symbol_pw)

311425221
%%#!%%@@$#@


In [22]:
create_password_creator.__code__.co_varnames

('s', 'create_password')

In [23]:
s = 'abcdefghij'

def get_3(x):
    return x[3]

def get_5(x):
    return x[5]

get_3(s)    

'd'

In [24]:
get_5(s)

'f'

In [25]:
import operator

In [28]:
get_3 = operator.itemgetter(3)
get_3(s)

'd'

In [29]:
def my_itemgetter(index):
    def inner(data):
        return data[index]
    return inner

In [30]:
get_3 = my_itemgetter(3)
get_3(s)

'd'

In [35]:
words = 'This isnt another bunch off words Python class teaching today'.split()

In [36]:
sorted(words)

['Python',
 'This',
 'another',
 'bunch',
 'class',
 'isnt',
 'off',
 'teaching',
 'today',
 'words']

In [38]:
sorted(words, key=operator.itemgetter(2,1))

['teaching',
 'class',
 'today',
 'off',
 'This',
 'isnt',
 'bunch',
 'another',
 'words',
 'Python']

In [39]:
sorted(words, key=my_itemgetter(2))

['class',
 'teaching',
 'today',
 'off',
 'This',
 'isnt',
 'bunch',
 'another',
 'words',
 'Python']

In [40]:
import random

def create_password_creator(s):
    def create_password(n):
        output = ''
        for index in range(n):
            output += random.choice(s)
        return output
    return create_password

create_number_pw = create_password_creator('12345')
create_symbol_pw = create_password_creator('!@#$%')

new_number_pw = create_number_pw(9)
new_symbol_pw = create_symbol_pw(11)

print(new_number_pw)
print(new_symbol_pw)

115415142
$$@!@@@$@%$


In [42]:
create_number_pw.__code__.co_varnames

('n', 'output', 'index')

In [44]:
create_number_pw.__code__.co_freevars    # from the enclosing function

('s',)

In [45]:
create_password_creator.__code__.co_cellvars    # used by the inner function

('s',)

In [46]:
# how many times did we run hello?

counter = 0

def outer():
    def hello(name):
        global counter
        counter += 1
        return f'{counter}: Hello, {name}!'
    return hello

x = outer()
x('world')

'1: Hello, world!'

In [47]:
x('Reuven')

'2: Hello, Reuven!'

In [51]:
# how many times did we run hello?

def outer():
    counter = 0
    def hello(name):
        nonlocal counter
        counter += 1
        return f'{counter}: Hello, {name}!'
    return hello

x = outer()
print(x('a'))
print(x('b'))
print(x('c'))

1: Hello, a!
2: Hello, b!
3: Hello, c!


In [53]:
# how many times did we run hello?

def greet(greeting):
    counter = 0
    def inner(name):
        nonlocal counter
        counter += 1
        return f'{counter}: {greeting}, {name}!'
    return inner

hello = greet('hello')
goodbye = greet('goodbye')

In [54]:
hello('world')

'1: hello, world!'

In [55]:
hello('again')

'2: hello, again!'

In [56]:
hello('again again')

'3: hello, again again!'

In [57]:
goodbye('everyone')

'1: goodbye, everyone!'

In [58]:
def hello(name):
    s = f'Hello, {name}!'
    return s

In [59]:
import dis

In [60]:
dis.dis(hello)

  1           0 RESUME                   0

  2           2 LOAD_CONST               1 ('Hello, ')
              4 LOAD_FAST                0 (name)
              6 FORMAT_VALUE             0
              8 LOAD_CONST               2 ('!')
             10 BUILD_STRING             3
             12 STORE_FAST               1 (s)

  3          14 LOAD_FAST                1 (s)
             16 RETURN_VALUE


In [61]:
def hello(name):
    global s
    s = f'Hello, {name}!'
    return s

In [62]:
dis.dis(hello)

  1           0 RESUME                   0

  3           2 LOAD_CONST               1 ('Hello, ')
              4 LOAD_FAST                0 (name)
              6 FORMAT_VALUE             0
              8 LOAD_CONST               2 ('!')
             10 BUILD_STRING             3
             12 STORE_GLOBAL             0 (s)

  4          14 LOAD_GLOBAL              0 (s)
             24 RETURN_VALUE


In [63]:
s = 'hello out there'

def hello(name):
    return s

In [64]:
dis.dis(hello)

  3           0 RESUME                   0

  4           2 LOAD_GLOBAL              0 (s)
             12 RETURN_VALUE


In [65]:
def hello():
    print('hello')

In [66]:
dis.dis(hello)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + print)
             12 LOAD_CONST               1 ('hello')
             14 CALL                     1
             22 POP_TOP
             24 RETURN_CONST             0 (None)


In [67]:
def hello(name):
    global name
    return f'Hello, {name}'

SyntaxError: name 'name' is parameter and global (3118209731.py, line 2)

In [68]:
def outer(x):
    def inner(y):
        return f'In inner, {x=} and {y=}'
    return inner

In [69]:
dis.dis(outer)

              0 MAKE_CELL                0 (x)

  1           2 RESUME                   0

  2           4 LOAD_CLOSURE             0 (x)
              6 BUILD_TUPLE              1
              8 LOAD_CONST               1 (<code object inner at 0x105196950, file "/var/folders/d9/v8tsklln4477fll05wkgcpth0000gn/T/ipykernel_49084/268129923.py", line 2>)
             10 MAKE_FUNCTION            8 (closure)
             12 STORE_FAST               1 (inner)

  4          14 LOAD_FAST                1 (inner)
             16 RETURN_VALUE

Disassembly of <code object inner at 0x105196950, file "/var/folders/d9/v8tsklln4477fll05wkgcpth0000gn/T/ipykernel_49084/268129923.py", line 2>:
              0 COPY_FREE_VARS           1

  2           2 RESUME                   0

  3           4 LOAD_CONST               1 ('In inner, x=')
              6 LOAD_DEREF               1 (x)
              8 FORMAT_VALUE             2 (repr)
             10 LOAD_CONST               2 (' and y=')
             

In [70]:
def outer():
    x = 100
    def inner():
        nonlocal x
        x = 200
    return inner

dis.dis(outer)    

              0 MAKE_CELL                1 (x)

  1           2 RESUME                   0

  2           4 LOAD_CONST               1 (100)
              6 STORE_DEREF              1 (x)

  3           8 LOAD_CLOSURE             1 (x)
             10 BUILD_TUPLE              1
             12 LOAD_CONST               2 (<code object inner at 0x10533adb0, file "/var/folders/d9/v8tsklln4477fll05wkgcpth0000gn/T/ipykernel_49084/2014286018.py", line 3>)
             14 MAKE_FUNCTION            8 (closure)
             16 STORE_FAST               0 (inner)

  6          18 LOAD_FAST                0 (inner)
             20 RETURN_VALUE

Disassembly of <code object inner at 0x10533adb0, file "/var/folders/d9/v8tsklln4477fll05wkgcpth0000gn/T/ipykernel_49084/2014286018.py", line 3>:
              0 COPY_FREE_VARS           1

  3           2 RESUME                   0

  5           4 LOAD_CONST               1 (200)
              6 STORE_DEREF              0 (x)
              8 RETURN_CONST  

In [71]:
x = 100

for i in range(5):
    x = i ** 2

print(x)    

16


In [72]:
x = 100

[x
 for x in range(5)]

x 

100

In [73]:
def myfunc():
    global x
    [x
     for x in range(5)]


In [74]:
dis.dis(myfunc)

  1           0 RESUME                   0

  3           2 LOAD_GLOBAL              1 (NULL + range)
             12 LOAD_CONST               1 (5)
             14 CALL                     1

  2          22 GET_ITER
             24 LOAD_FAST_AND_CLEAR      0 (x)
             26 SWAP                     2
             28 BUILD_LIST               0
             30 SWAP                     2
        >>   32 FOR_ITER                 4 (to 44)

  3          36 STORE_FAST               0 (x)

  2          38 LOAD_FAST                0 (x)
             40 LIST_APPEND              2
             42 JUMP_BACKWARD            6 (to 32)
        >>   44 END_FOR
             46 SWAP                     2
             48 STORE_FAST               0 (x)
             50 POP_TOP
             52 RETURN_CONST             0 (None)
        >>   54 SWAP                     2
             56 POP_TOP
             58 SWAP                     2
             60 STORE_FAST               0 (x)
             62 RERA

In [75]:
s = '[x for x in range(5)]'



In [76]:
dis.dis(s)

  0           0 RESUME                   0

  1           2 PUSH_NULL
              4 LOAD_NAME                0 (range)
              6 LOAD_CONST               0 (5)
              8 CALL                     1
             16 GET_ITER
             18 LOAD_FAST_AND_CLEAR      0 (x)
             20 SWAP                     2
             22 BUILD_LIST               0
             24 SWAP                     2
        >>   26 FOR_ITER                 4 (to 38)
             30 STORE_FAST               0 (x)
             32 LOAD_FAST                0 (x)
             34 LIST_APPEND              2
             36 JUMP_BACKWARD            6 (to 26)
        >>   38 END_FOR
             40 SWAP                     2
             42 STORE_FAST               0 (x)
             44 RETURN_VALUE
        >>   46 SWAP                     2
             48 POP_TOP
             50 SWAP                     2
             52 STORE_FAST               0 (x)
             54 RERAISE                  0
Except

In [78]:
def myfunc():
    global x
    [x
     for x in range(5)]
    print(x)


In [79]:
myfunc()

100


In [80]:
# Dispatch table



In [ ]:
def a():
    return 'a'

def b():
    return 'b'

choice = 